# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Estimated memory usage (per MPI process):
        nonlocal projectors  :   3.42MiB
        single ψ             :    486KiB
        single ρ             :    108KiB
        peak memory (SCF)    :   5.99MiB
    
    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : Pse

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Estimated memory usage (per MPI process):
        nonlocal projectors  :   88.3MiB
        single ψ             :   8.83MiB
        single ρ             :    432KiB
        peak memory (SCF)    :    118MiB
    
    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : Ps

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -9.355358374704                   -1.10    6.4    178ms
  2   -9.356810763815       -2.84       -1.43    1.0   88.1ms
  3   -9.357074741554       -3.58       -2.78    2.4    104ms
  4   -9.357110940185       -4.44       -3.02    8.0    225ms
  5   -9.357111227969       -6.54       -3.14    1.0   85.6ms
  6   -9.357111372492       -6.84       -3.29    1.0   84.1ms
  7   -9.357111455793       -7.08       -3.45    1.0   84.4ms
  8   -9.357111488210       -7.49       -3.61    1.0   84.9ms
  9   -9.357111504378       -7.79       -3.90    1.1   86.1ms
 10   -9.357111507697       -8.48       -4.15    1.2   87.6ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -18.74740684555                   -0.97    6.6    1.04s
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     3
│     1
│     1
│     7
│     1
│     7
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:80
  2   -18.75907405262       -1.93       -1.32    2.8    364ms
  3   -18.79158879671       -1.49       -2.17    4.5    372ms
  4   -18.79259745186       -3.00       -2.50    5.0    445ms
  5   -18.79260436880       -5.16       -3.29    2.6    317ms
  6   -18.79260623337       -5.73       -3.58    5.2    454ms
  7   -18.79260638341       -6.82       -3.95    1.6    227ms
  8   -18.79260639283       -8.03       -4.44    2.9    252ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -37.55027280151                   -0.84    9.1    1.74s
  2   -37.55781317873       -2.12       -1.22    2.0    776ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     13
│      8
│     14
│     16
│     18
│     18
│     11
│     14
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:80
  3   -37.56450653673       -2.17       -2.11   14.0    2.11s
  4   -37.56481245649       -3.51       -2.32   10.2    1.50s
  5   -37.56494262488       -3.89       -3.09    1.5    809ms
  6   -37.56494677326       -5.38       -3.48    6.9    1.40s
  7   -37.56494967837       -5.54       -3.93    3.0    903ms
  8   -37.56494986363       -6.73       -4.46    4.1    1.15s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -9.355428227663                   -1.10    5.9    708ms
  2   -9.356856836375       -2.85       -1.90    1.0    229ms
  3   -9.357054257302       -3.70       -2.41    5.9    152ms
  4   -9.357098082065       -4.36       -2.68    2.9    104ms
  5   -9.357111217296       -4.88       -3.78    1.0   75.3ms
  6   -9.357111431923       -6.67       -3.85    5.5    148ms
  7   -9.357111489773       -7.24       -4.13    1.4   76.3ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     11
│      2
│     16
│      5
│     18
│     12
│     11
│      5
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:80
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -37.51560153369                   -0.84   10.0    2.26s
  2   -37.55281372279       -1.43       -1.56    3.0    730ms
  3   -29.38234152432   +    0.91       -0.61   11.0    1.70s
  4   -37.08889064507        0.89       -1.21    7.8    1.59s
  5   -37.49211129596       -0.39       -1.42    3.2    1.00s
  6   -37.54129546608       -1.31       -1.81    2.5    913ms
  7   -36.82568168317   +   -0.15       -1.13    6.0    1.75s
  8   -37.55866154015       -0.13       -2.11    5.2    1.15s
  9   -37.56203429940       -2.47       -2.07    2.8    949ms
 10   -37.56334446709       -2.88       -2.35    1.9    805ms
 11   -37.56456972761       -2.91   

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.